# False Information

In [15]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from akvmodel import *
from pgml import *
from tabulate import tabulate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
P = "p"

In [17]:
def build_pgml_models_from_akv(akv_model):
    # Build relation
    relation = [
        [akv_model.influence_graph[j][i] for j in range(akv_model.number_of_agents)]
        for i in range(akv_model.number_of_steps)
    ]
    # Remove self-loops
    for i in range(10):
        relation[i][i] = 0

    # Create a PGML model for each step
    pgml_models = []
    for i in range(len(akv_model.states)):
        pgml_models += [
            Model(
                akv_model.number_of_agents,
                relation,
                {"p": akv_model.states[i][0].tolist()},
                {"p": akv_model.states[i][1].tolist()},
            )
        ]

    return pgml_models

In [18]:
def bd12(model, world):
    table = [
        [f"v1([] p, w{world})", Box(Variable("p")).valuation1(model, world)],
        [f"v1(<> p, w{world})", Diamond(Variable("p")).valuation1(model, world)],
        [f"v2([] p, w{world})", Box(Variable("p")).valuation2(model, world)],
        [f"v2(<> p, w{world})", Diamond(Variable("p")).valuation2(model, world)],
    ]
    print(tabulate(table))

## Example 1: Disproving information with one extra agent

All agents trust each other by $0.5$ and believe the propostion by $(0.9, 0.1)$.
Then, with one more agent that agent $0$ trusts by $1$ and that believes the proposition
by $(0.1, 0.9)$.

In [19]:
num_agents = 9
relation = [
    [0.5 if i != j else 0 for j in range(num_agents)] for i in range(num_agents)
]
valuation1 = {P: [0.9 for _ in range(num_agents)]}
valuation2 = {P: [1 - valuation1["p"][i] for i in range(num_agents)]}
model = Model(num_agents, relation, valuation1, valuation2)

bd12(model, 0)

------------  ---
v1([] p, w0)  1
v1(<> p, w0)  0.5
v2([] p, w0)  0.1
v2(<> p, w0)  0.1
------------  ---


In [20]:
num_agents = 10
relation = [
    [0.5 if i != j else 0 for j in range(num_agents)] for i in range(num_agents)
]
relation[0][9] = 1
valuation1 = {P: [0.9 for _ in range(num_agents)]}
valuation1[P][9] = 0.1
valuation2 = {P: [1 - valuation1["p"][i] for i in range(num_agents)]}
model = Model(num_agents, relation, valuation1, valuation2)

bd12(model, 0)

------------  ---
v1([] p, w0)  0.1
v1(<> p, w0)  0.5
v2([] p, w0)  0.9
v2(<> p, w0)  0.1
------------  ---


## Example 2: Prove information with one extra agent

All agents trust each other by $0.5$ and believe the propostion by $(0.1, 0.9)$.
Then, with one more agent that agent $0$ trusts by $1$ and that believes the proposition
by $(0.9, 0.1)$.

In [21]:
num_agents = 9
relation = [
    [0.5 if i != j else 0 for j in range(num_agents)] for i in range(num_agents)
]
valuation1 = {P: [0.1 for _ in range(num_agents)]}
valuation2 = {P: [1 - valuation1["p"][i] for i in range(num_agents)]}
model = Model(num_agents, relation, valuation1, valuation2)

bd12(model, 0)

------------  ---
v1([] p, w0)  0.1
v1(<> p, w0)  0.1
v2([] p, w0)  0.5
v2(<> p, w0)  1
------------  ---


In [22]:
num_agents = 10
relation = [
    [0.5 if i != j else 0 for j in range(num_agents)] for i in range(num_agents)
]
relation[0][9] = 1
valuation1 = {P: [0.1 for _ in range(num_agents)]}
valuation1[P][9] = 0.9
valuation2 = {P: [1 - valuation1[P][i] for i in range(num_agents)]}
model = Model(num_agents, relation, valuation1, valuation2)
print(valuation1)
print(valuation2)
bd12(model, 0)

{'p': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.9]}
{'p': [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.09999999999999998]}
------------  ---
v1([] p, w0)  0.1
v1(<> p, w0)  0.9
v2([] p, w0)  0.5
v2(<> p, w0)  0.1
------------  ---


## Example 3: Positively influencing uninformed agents

In [23]:
num_agents = 9
relation = [
    [0.5 if i != j else 0 for j in range(num_agents)] for i in range(num_agents)
]
valuation1 = {P: [0 for _ in range(num_agents)]}
valuation2 = {P: [0 for _ in range(num_agents)]}
model = Model(num_agents, relation, valuation1, valuation2)

bd12(model, 0)

------------  -
v1([] p, w0)  0
v1(<> p, w0)  0
v2([] p, w0)  0
v2(<> p, w0)  0
------------  -


In [32]:
num_agents = 10
relation = [
    [0.5 if i != j else 0 for j in range(num_agents)] for i in range(num_agents)
]
for i in range(num_agents - 1):
    relation[i][num_agents - 1] = 1
valuation1 = {"p": [0 for _ in range(num_agents)]}
valuation1["p"][num_agents - 1] = 1
valuation2 = {"p": [0 for _ in range(num_agents)]}
valuation2["p"][num_agents - 1] = 0
model = Model(num_agents, relation, valuation1, valuation2)

bd12(model, 0)
bd12(model, 9)

------------  -
v1([] p, w0)  0
v1(<> p, w0)  1
v2([] p, w0)  0
v2(<> p, w0)  0
------------  -
------------  -
v1([] p, w9)  0
v1(<> p, w9)  0
v2([] p, w9)  0
v2(<> p, w9)  0
------------  -


## Example 3: Negatively influencing uninformed agents

In [29]:
num_agents = 9
relation = [
    [0.5 if i != j else 0 for j in range(num_agents)] for i in range(num_agents)
]
valuation1 = {P: [0 for _ in range(num_agents)]}
valuation2 = {P: [0 for _ in range(num_agents)]}
model = Model(num_agents, relation, valuation1, valuation2)

bd12(model, 0)

------------  -
v1([] p, w0)  0
v1(<> p, w0)  0
v2([] p, w0)  0
v2(<> p, w0)  0
------------  -


In [33]:
num_agents = 10
relation = [
    [0.5 if i != j else 0 for j in range(num_agents)] for i in range(num_agents)
]
for i in range(num_agents - 1):
    relation[i][num_agents - 1] = 1
valuation1 = {"p": [0 for _ in range(num_agents)]}
valuation1["p"][num_agents - 1] = 0
valuation2 = {"p": [0 for _ in range(num_agents)]}
valuation2["p"][num_agents - 1] = 1
model = Model(num_agents, relation, valuation1, valuation2)

bd12(model, 0)
bd12(model, 9)

------------  -
v1([] p, w0)  0
v1(<> p, w0)  0
v2([] p, w0)  1
v2(<> p, w0)  0
------------  -
------------  -
v1([] p, w9)  0
v1(<> p, w9)  0
v2([] p, w9)  0
v2(<> p, w9)  0
------------  -
